In [11]:
import SimpleITK as sitk
import os
import time
from glob import glob


def writeSlices(series_tag_values, new_img, i, out_dir):
    image_slice = new_img[:,:,i]
    writer = sitk.ImageFileWriter()
    writer.KeepOriginalImageUIDOn()

    # Tags shared by the series.
    list(map(lambda tag_value: image_slice.SetMetaData(tag_value[0], tag_value[1]), series_tag_values))

    # Slice specific tags.
    image_slice.SetMetaData("0008|0012", time.strftime("%Y%m%d")) # Instance Creation Date
    image_slice.SetMetaData("0008|0013", time.strftime("%H%M%S")) # Instance Creation Time

    # Setting the type to CT preserves the slice location.
    image_slice.SetMetaData("0008|0060", "CT")  # set the type to CT so the thickness is carried over

    # (0020, 0032) image position patient determines the 3D spacing between slices.
    image_slice.SetMetaData("0020|0032", '\\'.join(map(str,new_img.TransformIndexToPhysicalPoint((0,0,i))))) # Image Position (Patient)
    image_slice.SetMetaData("0020|0013", str(i)) # Instance Number

    # Write to the output directory and add the extension dcm, to force writing in DICOM format.
    writer.SetFileName(os.path.join(out_dir,'slice' + str(i).zfill(4) + '.dcm'))
    writer.Execute(image_slice)


def nifti2dicom_1file(in_dir, out_dir):
    """
    This function is to convert only one nifti file into dicom series

    `nifti_dir`: the path to the one nifti file
    `out_dir`: the path to output
    """

    os.makedirs(out_dir, exist_ok=True)

    new_img = sitk.ReadImage(in_dir) 
    modification_time = time.strftime("%H%M%S")
    modification_date = time.strftime("%Y%m%d")

    direction = new_img.GetDirection()
    series_tag_values = [("0008|0031",modification_time), # Series Time
                    ("0008|0021",modification_date), # Series Date
                    ("0008|0008","DERIVED\\SECONDARY"), # Image Type
                    ("0020|000e", "1.2.826.0.1.3680043.2.1125."+modification_date+".1"+modification_time), # Series Instance UID
                    ("0020|000d", "1.2.826.0.1.3680043.2.1125.1"+modification_date+modification_time), # Study Instance UID
                    ("0020|0037", '\\'.join(map(str, (direction[0], direction[3], direction[6],# Image Orientation (Patient)
                                                        direction[1],direction[4],direction[7])))),
                    ("0008|103e", "Created-Pycad")] # Series Description

    # Write slices to output directory
    list(map(lambda i: writeSlices(series_tag_values, new_img, i, out_dir), range(new_img.GetDepth())))

def nifti2dicom_mfiles(nifti_dir, out_dir=''):
    """
    This function is to convert multiple nifti files into dicom files

    `nifti_dir`: You enter the global path to all of the nifti files here.
    `out_dir`: Put the path to where you want to save all the dicoms here.

    PS: Each nifti file's folders will be created automatically, so you do not need to create an empty folder for each patient.
    """

    images = glob(nifti_dir + '/*.nii.gz')

    for image in images:
        o_path = out_dir + '/' + os.path.basename(image)[:-7]
        os.makedirs(o_path, exist_ok=True)

        nifti2dicom_1file(image, o_path)

In [12]:
nifti_folder = r'E:\Medical imaging\Liver segmentation\large set\volume_pt6'
output_folder = r'E:\Medical imaging\Liver segmentation\large set\dcom files\images'
patient_list=glob(nifti_folder + '/*') 
for patient in patient_list: 
    patient_name = os.path.basename(os.path.normpath(patient))
    output_path_name = os.path.join(output_folder, patient_name)
    os.mkdir(output_path_name)
    nifti2dicom_1file(patient, output_path_name)


In [9]:
nifti_folder = r'E:\Medical imaging\Liver segmentation\large set\segmentations - Copy - Copy'
output_folder = r'E:\Medical imaging\Liver segmentation\large set\dcom files\dup label'
patient_list=glob(nifti_folder + '/*') 
for patient in patient_list: 
    patient_name = os.path.basename(os.path.normpath(patient))
    output_path_name = os.path.join(output_folder, patient_name) 
    os.mkdir(output_path_name)
    nifti2dicom_1file(patient, output_path_name)

RuntimeError: Exception thrown in SimpleITK ImageFileWriter_Execute: D:\a\SimpleITK\SimpleITK\bld\ITK\Modules\IO\GDCM\src\itkGDCMImageIO.cxx:1229:
ITK ERROR: GDCMImageIO(00000291E1A3C5F0): A Floating point buffer was passed but the stored pixel type was not specified.This is currently not supported

In [10]:
import os
import SimpleITK as sitk
from glob import glob

# Function to convert a NIfTI file to DICOM format
def nifti2dicom_1file(nifti_file, output_folder):
    # Read the NIfTI image
    new_img = sitk.ReadImage(nifti_file)

    # Rescale and cast the image to an integer type (e.g., int16)
    rescaled_img = sitk.Cast(sitk.RescaleIntensity(new_img), sitk.sitkInt16)

    # Write slices to output directory
    for i in range(rescaled_img.GetDepth()):
        slice_img = rescaled_img[:, :, i]

        # Set up DICOM writer
        writer = sitk.ImageFileWriter()
        writer.SetFileName(os.path.join(output_folder, f'slice{i:04d}.dcm'))
        
        # Write the DICOM slice
        writer.Execute(slice_img)

# Paths to the NIfTI files and the output directory
nifti_folder = r'E:\Medical imaging\Liver segmentation\large set\segmentations - Copy'
output_folder = r'E:\Medical imaging\Liver segmentation\large set\dcom files\dup label'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Get the list of patients (NIfTI files)
patient_list = glob(nifti_folder + '/*')

# Iterate over each patient's folder, create an output folder, and convert NIfTI to DICOM
for patient in patient_list:
    patient_name = os.path.basename(os.path.normpath(patient))
    output_path_name = os.path.join(output_folder, patient_name)
    
    # Create a directory for the patient if it doesn't exist
    os.makedirs(output_path_name, exist_ok=True)
    
    # Convert the NIfTI file to DICOM format
    nifti2dicom_1file(patient, output_path_name)

print("Conversion complete!")


Conversion complete!


In [6]:
indir=r'E:\Medical imaging\Liver segmentation\large set\try newly\New folder\New folder'
outdir=r'E:\Medical imaging\Liver segmentation\large set\try newly\New folder\1'
nifti2dicom_1file(indir,outdir)

RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: D:\a\SimpleITK\SimpleITK\Code\IO\src\sitkImageReaderBase.cxx:96:
sitk::ERROR: Unable to open "E:\Medical imaging\Liver segmentation\large set\try newly\New folder\New folder" for reading.